In [1]:
using AutomotiveDrivingModels
using AutoScenes
using AutoViz
using NGSIM

INFO: Recompiling stale cache file /home/tim/.julia/lib/v0.5/AutoScenes.ji for module AutoScenes.


In [2]:
trajdata = load_trajdata(4)
scene = get!(Scene(), trajdata, 4000);

In [3]:
# structure = gen_scene_structure(scene, trajdata.roadway, create_shared_factors())
# overlays = SceneOverlay[Overwash(RGBA(1.0,1.0,1.0,0.7)), SceneStructureOverlay(structure)]
# render(scene, trajdata.roadway, overlays, cam=SceneFollowCamera(4.0))

In [4]:
# for NGSIM
const REGIONS = Dict{ASCIIString, SubSceneExtractParams}(
  "101A" => SubSceneExtractParams(VecSE2(1966395.000, 570900.000, deg2rad(138)), 100.0, 41.0),  
  "101B" => SubSceneExtractParams(VecSE2(1966501.608, 570804.793, 2.425), 100.0, 25.0),
  "80A"  => SubSceneExtractParams(VecSE2(1841794.208, 650650.640, deg2rad(-81)), 100.0, 32.0),
  "80B"  => SubSceneExtractParams(VecSE2(1841849.208, 650320.640, deg2rad(-81)), 100.0, 38.0),
  )

  likely near In[4]:2


Dict{String,AutoScenes.SubSceneExtractParams} with 4 entries:
  "80A"  => AutoScenes.SubSceneExtractParams(VecSE2({1841794.208, 650650.640}, …
  "101B" => AutoScenes.SubSceneExtractParams(VecSE2({1966501.608, 570804.793}, …
  "101A" => AutoScenes.SubSceneExtractParams(VecSE2({1966395.000, 570900.000}, …
  "80B"  => AutoScenes.SubSceneExtractParams(VecSE2({1841849.208, 650320.640}, …

In [5]:
# overlays = SceneOverlay[SubSceneExtractParamsOverlay(REGIONS["80A"])]
# render(scene, trajdata.roadway, overlays, cam=SceneFollowCamera(1.0))

In [6]:
# using Interact

# # x0 = 1966600.0 # 101
# # y0 = 570700.0

# x0 = 1841794.208 # 80
# y0 = 650650.640
# @manipulate for width in linspace(25.0,125.0,101),
#                 x in linspace(x0-500.0, x0+500.0, 201),
#                 y in linspace(y0-500.0, y0+500.0, 201),
#                 angle in linspace(-360.0, 360.0, 360*2+1),
#                 zoom in linspace(-3.0, 2.0, 101)
#     region = SubSceneExtractParams(VecSE2(x,  y, deg2rad(angle)), 100.0, width)
#     overlays = SceneOverlay[SubSceneExtractParamsOverlay(region)]
#     render(scene, trajdata.roadway, overlays, cam=StaticCamera(convert(VecE2, region.center), exp(zoom)))
# end

In [7]:
# region.center

In [8]:
factors = create_shared_factors()
dset = SceneStructureDataset(factors)

for trajdata_index in 1 : 6
    trajdata = load_trajdata(trajdata_index)
    frames = 1:40:nframes(trajdata)
    if startswith(splitdir(NGSIM.TRAJDATA_PATHS[trajdata_index])[2], "trajdata_i101")
        append!(dset, pull_scene_dataset(trajdata, REGIONS["101A"], frames=frames))
        append!(dset, pull_scene_dataset(trajdata, REGIONS["101B"], frames=frames))
    else
        append!(dset, pull_scene_dataset(trajdata, REGIONS["80A"], frames=frames))
        append!(dset, pull_scene_dataset(trajdata, REGIONS["80B"], frames=frames))
    end
end

length(dset)

1371

In [9]:
dat = PseudolikelihoodPrealloc(50);

In [10]:
# tic()
# calc_pseudolikelihood(dset, dat=dat)
# toc()

In [11]:
params = GradientStepParams(BatchSampler(dset))
params.grad_params.n_samples_monte_carlo_integration = 5
params.grad_params.n_samples_monte_carlo_pseudolikelihood = 5

learning_rate = 0.01
batch_size = 2
t_start = now()

iter = 0
while iter < typemax(Int)
    iter += 1
    
    params.learning_rate = learning_rate
    params.batch_size = batch_size
    parallel_step!(params)
    
    println("iter: ", iter)
    println("time: ", now() - t_start)
    if mod(iter, 5) == 1
        println("plogl: ", calc_pseudolikelihood(dset, dat=dat, scene=params.grad_params.scene, rec=params.grad_params.rec))
    end
    println("learning rate: ", learning_rate)
    println("batch_size:    ", batch_size)
    println("n_samples:     ", )
    println("weights: ")
    for ϕ in dset.factors
        println(ϕ.template.form, "  ", ϕ.weights)
    end
    println("")
    
    learning_rate *= 0.95
    batch_size += 1
end

iter: 1
time: 15103 milliseconds
plogl: 0.0
learning rate: 0.01
batch_size:    2
n_samples:     
weights: 
1  [-4.43216,-8.0,-8.0,-7.23949,-0.814754,1.0,-2.9099,0.811337,-0.204178,-3.30324,-7.76048,1.0,0.151251,0.313214,0.54468,-8.0,0.952374,1.0,1.0]
2  [-0.102183,-8.0,-5.22692,-0.077776,0.015017,1.0,-3.76083,0.441685,0.196299]
3  [-0.1333,0.436469,0.780003,0.363001,1.0]

iter: 2
time: 314987 milliseconds
learning rate: 0.0095
batch_size:    3
n_samples:     
weights: 
1  [-4.60133,-8.0,-8.0,-7.23467,-1.80863,1.0,-2.93478,1.0,-0.196604,-0.644106,-7.31528,1.0,0.108529,0.111201,0.577662,-8.0,0.721938,1.0,1.0]
2  [0.120515,-8.0,-5.19422,-0.10438,-0.0404975,1.0,-3.75742,0.492269,-0.38341]
3  [0.0414125,0.585011,0.770543,0.209614,1.0]



LoadError: LoadError: InterruptException:
 in evaluate(::AutoScenes.GraphFeatureTemplate, ::AutoScenes.GraphFeatureInstance) at /home/tim/.julia/v0.5/AutoScenes/src/features/core.jl:72
 in evaluate_dot(::AutoScenes.SharedFactor) at /home/tim/.julia/v0.5/AutoScenes/src/factors.jl:29
 in evaluate_dot!(::AutoScenes.SceneStructure, ::Array{AutoScenes.SharedFactor,1}, ::AutomotiveDrivingModels.AutoCore.Scene, ::AutomotiveDrivingModels.AutoCore.Roadway, ::AutomotiveDrivingModels.AutoCore.SceneRecord) at /home/tim/.julia/v0.5/AutoScenes/src/scene_structures.jl:92
 in calc_pseudolikelihood_gradient_component_ϕ(::AutoScenes.SharedFactor, ::Array{AutoScenes.SharedFactor,1}, ::AutomotiveDrivingModels.AutoCore.Scene, ::AutoScenes.SceneStructure, ::AutomotiveDrivingModels.AutoCore.Roadway, ::Int64, ::Array{Int64,1}, ::AutoScenes.GraphFeatureInstance, ::Int64, ::MersenneTwister, ::AutomotiveDrivingModels.AutoCore.SceneRecord) at /home/tim/.julia/v0.5/AutoScenes/src/learning.jl:572
 in calc_pseudolik

In [41]:
params = GradientStepParams(BatchSampler(dset))
params.grad_params.n_samples_monte_carlo_integration = 50
params.grad_params.n_samples_monte_carlo_pseudolikelihood = 50

learning_rate = 0.01
batch_size = 10
t_start = now()

iter = 0
while iter < typemax(Int)
    iter += 1
    
    params.learning_rate = learning_rate
    params.batch_size = batch_size
    step!(params)
    
    println("iter: ", iter)
    println("time: ", now() - t_start)
    if mod(iter, 5) == 1
        println("plogl: ", calc_pseudolikelihood(dset, dat=dat, scene=params.grad_params.scene, rec=params.grad_params.rec))
    end
    println("learning rate: ", learning_rate)
    println("batch_size:    ", batch_size)
    println("n_samples:     ", )
    println("weights: ")
    for ϕ in dset.factors
        println(ϕ.template.form, "  ", ϕ.weights)
    end
    println("")
    
    learning_rate *= 0.95
    batch_size += 1
end

1:   -0.009     0.001    -0.016     0.002    -0.003     0.000     0.004     0.000     0.002    -0.000    -0.010    -0.001    -0.003    -0.000    -0.001     0.000     0.001     0.000     0.041     0.002     0.024    -0.000    -0.019     0.001    -0.004    -0.004    -0.002    -0.001     0.002     0.000    -0.013     0.000    -0.001    -0.001     0.004    -0.000     0.005    -0.000  
2:    0.001    -0.012     0.002    -0.004    -0.000     0.003    -0.003     0.018    -0.018  
3: 
iter: 1
time: 290000 milliseconds
plogl: -100.54216178207922
learning rate: 0.01
batch_size:    10
n_samples:     
weights: 
1  [0.9914529423768134,0.8250582472158212,-8.0,-7.998202756579285,-0.0034957612130915305,1.0,-6.697781883159157,0.001245710218970501,1.0,0.08219980934165173,0.9899835087253473,-1.9751055749553639,-2.180796680281114,-0.01534772506639652,0.9992159966308126,0.028897966976149044,-0.5259337933526155,-0.0003404079371402058,-4.177403652066115,0.07270230214360382,-7.976462863421564,0.43311652631113

LoadError: LoadError: InterruptException:
while loading In[41], in expression starting on line 10

# Learned Marginal Distribution

In [15]:
# using PGFPlots

# g = GroupPlot(1, 3, groupStyle = "vertical sep = 1cm")
# push!(g, Plots.Linear(collect(1:10), randn(10)))
# push!(g, Plots.Linear(collect(1:10), randn(10)))
# push!(g, Plots.Linear(collect(1:10), randn(10)))
# g

! LaTeX Error: File `luatex85.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
   
l.2 \documentclass
                [tikz]{standalone}
End of file on the terminal!



Here is how much of LuaTeX's memory you used:
 11 strings out of 494693
 100000,89155 words of node,token memory allocated 188 words of node memory still in use:
   31 glue_spec, 1 dir nodes
   avail lists: 2:12,3:1
 3851 multiletter control sequences out of 65536+600000
 14 fonts using 562767 bytes
 10i,0n,7p,45b,8s stack positions out of 5000i,500n,10000p,200000b,100000s
!  ==> Fatal error occurred, no output PDF file produced!

Error saving as SVG


ErrorException: LaTeX error

In [16]:
# using PGFPlots

# function _cycle_linear_plot!(xarr::Vector{Float64}, yarr::Vector{Float64})
#     append!(yarr, [0.0, 0.0, yarr[1]])
#     append!(xarr, [xarr[end], xarr[1], xarr[1]])
#     nothing
# end
# function plot_marginal_roadfactor(ϕ_road::SharedFactor;
#     nbins_v::Int = 101,
#     nbins_t::Int = 101,
#     nbins_ϕ::Int = 101,
#     )

#     pts_v = collect(linspace(AutoScenes.BOUNDS_V[1], AutoScenes.BOUNDS_V[2], nbins_v))
#     pts_t = collect(linspace(-3.0, 3.0, nbins_t))
#     pts_ϕ = collect(linspace(AutoScenes.BOUNDS_ϕ[1], AutoScenes.BOUNDS_ϕ[2], nbins_ϕ))
#     v_values = zeros(Float64, nbins_v)
#     t_values = zeros(Float64, nbins_t)
#     ϕ_values = zeros(Float64, nbins_ϕ)
#     tot = 0.0

#     for (j,t) in enumerate(pts_t)
#         AutoScenes._set_and_standardize!(ϕ_road.template, t, 1)
#         for (i,v) in enumerate(pts_v)
#             AutoScenes._set_and_standardize!(ϕ_road.template, v, 2)
#             for (k,ϕ) in enumerate(pts_ϕ)
#                 AutoScenes._set_and_standardize!(ϕ_road.template, ϕ, 3)
                
#                 e = evaluate_dot(ϕ_road)
#                 v_values[i] += e
#                 t_values[j] += e
#                 ϕ_values[k] += e
#                 tot += e
#             end
#         end
#     end

#     # produce a probability density
#     v_values ./= tot
#     t_values ./= tot
#     ϕ_values ./= tot

#     plot_width = "20cm"
#     plot_height = "4cm"
#     style = "solid, thick, mark=none, black"

#     _cycle_linear_plot!(pts_v, v_values)
#     _cycle_linear_plot!(pts_t, t_values)
#     _cycle_linear_plot!(pts_ϕ, ϕ_values)

#     g = GroupPlot(1, 3, groupStyle = "vertical sep = 1cm")
#     push!(g, Axis(PGFPlots.Plots.Linear(pts_v, v_values, style=style*", fill=monokai3"), xlabel="speed [ft/s]", ylabel="pdf", width=plot_width, height=plot_height, enlargelimits="false"))
#     push!(g, Axis(PGFPlots.Plots.Linear(pts_t, t_values, style=style*", fill=monokai4"), xlabel="lane centerline offset [ft]", ylabel="pdf", width=plot_width, height=plot_height, enlargelimits="false"))
#     push!(g, Axis(PGFPlots.Plots.Linear(rad2deg(pts_ϕ), ϕ_values, style=style*", fill=monokai5"), xlabel="lane-relative heading [deg]", ylabel="pdf", width=plot_width, height=plot_height, enlargelimits="false"))
#     g
# end

# plot_marginal_roadfactor(dset.factors[1])

[0.009587413726315342,0.009583849385465516,0.009580583101230939,0.00957761487361168,0.00957494470260762,0.00957257258821884,0.009570498530445237,0.009568722529287057,0.009567244584744047,0.009566064696816218,0.009565182865503886,0.00956459909080668,0.009564313372724719,0.009564325711258008,0.009564636106406564,0.009565244558170463,0.009566151066549637,0.009567355631543973,0.009568858253153635,0.009570658931378593,0.009572757666218722,0.009575154457674128,0.009577849305744927,0.0095808422104309,0.009584133171732174,0.00958772218964863,0.009591609264180387,0.009595794395327446,0.009600277583089744,0.009605058827467278,0.009610138128460125,0.009615515486068248,0.00962119090029157,0.009627164371130285,0.009633435898584067,0.009640005482653325,0.009646873123337672,0.009654038820637399,0.009661502574552326,0.009669264385082544,0.009677324252228064,0.009685682175988789,0.0096943381563648,0.009703292193355962,0.009712544286962561,0.009722094437184307,0.009731942644021494,0.009742088907473762,0

ErrorException: LaTeX error